<a href="https://colab.research.google.com/github/TheoBacqueyrisse/graph-neural-networks/blob/main/GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Graph Neural Network Architecture**

In [ ]:
# Let us first clone the GitHub repository
%%capture
!git clone https://github.com/TheoBacqueyrisse/Graph-Neural-Networks.git

In [ ]:
# Install dependencies
%%capture
%cd /content/Graph-Neural-Networks
!pip install -r requirements.txt

In [ ]:
from utils import *

## GNN Module Architecture

In [ ]:
EMBEDDING_SIZE = 32

class GNN(torch.nn.Module):
    def __init__(self):
      super(GNN, self).__init__()

      # Care about the design of the NN here
      self.initial_conv = GATConv(in_channels = 1, out_channels = EMBEDDING_SIZE)
      self.conv_layer1 = GATConv(in_channels = EMBEDDING_SIZE, out_channels = EMBEDDING_SIZE)
      self.conv_layer2 = GATConv(in_channels = EMBEDDING_SIZE, out_channels = EMBEDDING_SIZE)

      self.pooling = gap
      self.out = Linear(in_features = EMBEDDING_SIZE, out_features = 1)

    def forward(self, x, edge_index, edge_attribute, batch_index):


      y = self.initial_conv(x, edge_index, edge_attribute)
      y = F.sigmoid(y)
      y = F.dropout(y, p = 0.2)

      y = self.conv_layer1(y, edge_index, edge_attribute)
      y = F.sigmoid(y)
      y = F.dropout(y, p = 0.1)

      y = self.conv_layer2(y, edge_index, edge_attribute)
      y = F.sigmoid(y)

      y = self.pooling(y, batch_index)

      out = self.out(y)

      return out, y

model = GNN()
print(model)

GNN(
  (initial_conv): GATConv(1, 32, heads=1)
  (conv_layer1): GATConv(32, 32, heads=1)
  (conv_layer2): GATConv(32, 32, heads=1)
  (out): Linear(in_features=32, out_features=1, bias=True)
)


## Configuration

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

NUM_EPOCHS = 100

loss_function = L1Loss()

optimizer = Adam(params = model.parameters(), lr = 0.003)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, min_lr=0.00001)

In [ ]:
NB_GRAPHS_PER_BATCH = 64

train = ZINC('/content/Graph-Neural-Networks/data', split = 'train')
train = train[train.y > -10] # Drop Outliers

val = ZINC('/content/Graph-Neural-Networks/data', split = 'val')

test = ZINC('/content/Graph-Neural-Networks/data', split = 'test')

train_loader = DataLoader(train,
                          batch_size = NB_GRAPHS_PER_BATCH,
                          shuffle = True)

val_loader = DataLoader(val,
                        batch_size = NB_GRAPHS_PER_BATCH,
                        shuffle = False)

test_loader = DataLoader(test,
                         batch_size = NB_GRAPHS_PER_BATCH,
                         shuffle = False)

print("Number of Batches in Train Loader :", len(train_loader))
print("Number of Batches in Val Loader :", len(val_loader))
print("Number of Batches in Test Loader :", len(test_loader))

Number of Batches in Train Loader : 3433
Number of Batches in Val Loader : 382
Number of Batches in Test Loader : 79


## Train and Test Functions 🚀

In [ ]:
def train(train_loader, val_loader):
  for epoch in range(NUM_EPOCHS+1):

    model.train()
    tot_train_loss = 0.0

    for batch in train_loader:

      # Use GPU
      batch.to(device)

      # Set Gradient values to 0
      optimizer.zero_grad()

      pred, y = model(batch.x.float(), batch.edge_index, batch.edge_attr, batch.batch)

      # Compute Loss and Gradients
      loss = loss_function(pred, batch.y.view(-1, 1).float())
      loss.backward()
      tot_train_loss += loss.item()

      optimizer.step()

    average_train_loss = tot_train_loss / len(train_loader)

    model.eval()
    with torch.no_grad():
        tot_val_loss = 0.0

        for val_batch in val_loader:
            val_batch.to(device)

            val_pred, val_y = model(val_batch.x.float(), val_batch.edge_index, val_batch.edge_attr, val_batch.batch)
            val_loss = loss_function(val_pred, val_batch.y.view(-1, 1).float())

            tot_val_loss += val_loss.item()

        average_val_loss = tot_val_loss / len(val_loader)

    scheduler.step(average_val_loss)

    # if epoch % 10 == 0:
    print(f"Epoch {epoch} -> Train Loss: {average_train_loss:.4f} - Val Loss: {average_val_loss:.4f}")


def test(test_loader):
  model.eval()
  with torch.no_grad():
      tot_test_loss = 0.0

      for test_batch in test_loader:
          test_batch.to(device)

          test_pred, test_y = model(test_batch.x.float(), test_batch.edge_index, test_batch.edge_attr, test_batch.batch)
          test_loss = loss_function(test_pred, test_batch.y.view(-1, 1).float())

          tot_test_loss += test_loss.item()

      average_test_loss = tot_test_loss / len(test_loader)

  print(f"Test Loss: {average_test_loss:.4f}")

## Model Training and Evaluation ⚡



In [ ]:
train(train_loader, val_loader)

Epoch 0 -> Train Loss: 1.1525 - Val Loss: 1.1180
Epoch 1 -> Train Loss: 1.0527 - Val Loss: 1.0269
Epoch 2 -> Train Loss: 1.0051 - Val Loss: 1.0247
Epoch 3 -> Train Loss: 0.9483 - Val Loss: 0.9431
Epoch 4 -> Train Loss: 0.9219 - Val Loss: 0.9213
Epoch 5 -> Train Loss: 0.9083 - Val Loss: 0.9290
Epoch 6 -> Train Loss: 0.8952 - Val Loss: 0.8971
Epoch 7 -> Train Loss: 0.8796 - Val Loss: 0.9044
Epoch 8 -> Train Loss: 0.8695 - Val Loss: 0.8915
Epoch 9 -> Train Loss: 0.8622 - Val Loss: 0.8833
Epoch 10 -> Train Loss: 0.8543 - Val Loss: 0.8593
Epoch 11 -> Train Loss: 0.8437 - Val Loss: 0.8487
Epoch 12 -> Train Loss: 0.8335 - Val Loss: 0.8366
Epoch 13 -> Train Loss: 0.8145 - Val Loss: 0.8168
Epoch 14 -> Train Loss: 0.7940 - Val Loss: 0.7989
Epoch 15 -> Train Loss: 0.7875 - Val Loss: 0.7925
Epoch 16 -> Train Loss: 0.7836 - Val Loss: 0.7948
Epoch 17 -> Train Loss: 0.7806 - Val Loss: 0.7945
Epoch 18 -> Train Loss: 0.7786 - Val Loss: 0.7879
Epoch 19 -> Train Loss: 0.7736 - Val Loss: 0.7911
Epoch 20 -

In [ ]:
test(test_loader)

Test Loss: 0.6773
